# 부록 10.2: 도구 사용

- [수업](#lesson)
- [연습 문제](#exercises)
- [예시 실습장](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY variable from the IPython store
%store -r API_KEY

client = anthropic.Anthropic(api_key=API_KEY)

# Rewrittten to call Claude 3 Sonnet, which is generally better at tool use, and include stop_sequences
def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages,
        stop_sequences=stop_sequences
    )
    return message.content[0].text

---

## 수업

처음에는 개념적으로 복잡해 보일 수 있지만, 도구 사용(일명 함수 호출)은 실제로는 매우 간단합니다! 여러분은 이미 도구 사용을 구현하는 데 필요한 모든 기술을 알고 있습니다. 이는 실제로는 대체와 프롬프트 연결의 조합일 뿐입니다.

이전 대체 연습에서는 텍스트를 프롬프트에 대체했습니다. 도구 사용에서는 도구나 함수 결과를 프롬프트에 대체합니다. Claude는 말 그대로 도구와 함수를 호출하거나 접근할 수 없습니다. 대신 Claude가:
1. 호출하고자 하는 도구 이름과 인수를 출력
2. 도구가 호출되는 동안 추가 응답 생성을 중단
3. 그 다음 도구 결과가 추가된 상태로 다시 프롬프트

함수 호출은 Claude의 능력을 확장하고 Claude가 훨씬 더 복잡하고 다단계 작업을 처리할 수 있게 하기 때문에 유용합니다.
Claude에게 제공할 수 있는 함수의 몇 가지 예시:
- 계산기
- 단어 카운터
- SQL 데이터베이스 쿼리 및 데이터 검색
- 날씨 API

다음 두 가지 요소를 결합하여 Claude가 도구 사용을 하도록 할 수 있습니다:

1. 시스템 프롬프트: 여기서 Claude에게 도구 사용의 개념과 접근할 수 있는 도구의 상세한 설명 목록을 제공합니다
2. Claude의 도구 사용 요청을 조정하고 실행하는 제어 로직

### 도구 사용 로드맵

*이 수업은 현재의 도구 사용 형식을 가르칩니다. 하지만 가까운 미래에 도구 사용 기능을 업데이트하고 개선할 예정입니다:*
* *함수 정의 및 호출을 위한 더 간소화된 형식*
* *더 견고한 오류 처리 및 엣지 케이스 적용 범위*
* *나머지 API와의 더 긴밀한 통합*
* *특히 더 복잡한 도구 사용 작업에 대한 더 나은 신뢰성과 성능*

### 예시

Claude에서 도구 사용을 활성화하려면 시스템 프롬프트부터 시작합니다. 이 특별한 도구 사용 시스템 프롬프트에서 Claude에게 다음을 알려줍니다:
* 도구 사용의 기본 전제와 그것이 수반하는 것
* Claude가 주어진 도구를 어떻게 호출하고 사용할 수 있는지
* 이 특정 시나리오에서 접근할 수 있는 도구의 상세한 목록 

다음은 Claude에게 도구 사용을 설명하는 시스템 프롬프트의 첫 번째 부분입니다. 시스템 프롬프트의 이 부분은 도구 사용을 위해 Claude를 프롬프트하는 모든 인스턴스에서 일반화할 수 있습니다. Claude에게 제공하는 도구 호출 구조(`<function_calls> [...] </function_calls>`)는 Claude가 특별히 사용하도록 훈련된 구조이므로, 이를 고수하는 것을 권장합니다.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

다음은 이 특정 상황에서 Claude가 접근할 수 있는 정확한 도구를 정의하는 시스템 프롬프트의 두 번째 부분입니다. 이 예시에서는 Claude에게 두 개의 피연산자와 하나의 연산자라는 세 개의 매개변수를 받는 계산기 도구를 제공할 것입니다. 

그 다음 시스템 프롬프트의 두 부분을 결합합니다.

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

이제 `calculator` 도구 사용이 필요한 질문을 Claude에게 제공할 수 있습니다. Claude가 함수를 호출하는지와 언제 호출하는지를 감지하기 위해 `stop_sequences`에서 `<function_calls\>`를 사용할 것입니다.

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

이제 Claude의 함수 호출에서 매개변수를 추출하고 Claude를 대신해서 실제로 함수를 실행할 수 있습니다.

먼저 함수의 코드를 정의하겠습니다.

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

그 다음 Claude의 함수 호출 응답에서 매개변수를 추출할 것입니다. 모든 매개변수가 존재한다면, 계산기 도구를 실행합니다.

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

이제 결과가 있으니, Claude에게 다시 전달할 때 Claude가 그 결과가 어떤 도구와 관련된 것인지 이해할 수 있도록 그 결과를 적절히 형식화해야 합니다. Claude가 인식하도록 훈련된 설정된 형식이 있습니다:
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

아래 셀을 실행하여 위의 도구 결과를 이 구조로 형식화하세요.

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

이제 우리가 해야 할 일은 이전과 같은 메시지 체인에 결과를 추가하여 이 결과를 Claude에게 다시 보내는 것뿐입니다!

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

전체 도구 사용 체인을 끝에서 끝까지 실행한 것을 축하합니다!

이제 주어진 도구를 전혀 사용할 필요가 없는 질문을 Claude에게 제공하면 어떻게 될까요?

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

성공입니다! 보시다시피, Claude는 함수가 필요하지 않을 때 호출하지 않는 것을 알고 있었습니다.

위의 내용을 변경하지 않고 수업 프롬프트를 실험해보고 싶다면, 수업 노트북의 맨 아래로 스크롤하여 [**예시 실습장**](#example-playground)을 방문하세요.

---

## 연습 문제
- [연습 문제 10.2.1 - SQL](#exercise-1021---SQL)

### 연습 문제 10.2.1 - SQL
이 연습에서는 세계에서 가장 작은 "데이터베이스"를 쿼리하고 작성하기 위한 도구 사용 프롬프트를 작성할 것입니다. 다음은 초기화된 데이터베이스인데, 실제로는 그냥 딕셔너리입니다.

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

그리고 다음은 데이터베이스에서 쓰기와 읽기를 하는 함수들의 코드입니다.

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

연습 문제를 해결하려면, 위의 `system_prompt_tools_specific_tools`와 같은 시스템 프롬프트를 정의하는 것부터 시작하세요. 각 함수에 대한 각 매개변수의 이름과 유형 및 설명과 함께 각 도구의 이름과 설명을 포함해야 합니다. 아래에 시작을 위한 일부 기본 구조를 제공했습니다.

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

준비가 되면, 아래 예시들에서 도구 정의 시스템 프롬프트를 시도해볼 수 있습니다. 아래 셀을 실행하기만 하면 됩니다!

In [ ]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print Claude's response
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

제대로 했다면, 함수 호출 메시지들이 `add_user`, `add_product`, `get_user`, 그리고 `get_product` 함수들을 올바르게 호출해야 합니다.

추가 점수를 위해, 일부 코드 셀을 추가하고 매개변수 파싱 코드를 작성하세요. 그 다음 Claude가 제공한 매개변수로 함수들을 호출하여 호출 후 "데이터베이스"의 상태를 확인하세요.

❓ 가능한 솔루션을 보고 싶다면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### 축하합니다!

도구 사용과 함수 호출을 배운 것을 축하합니다! 검색 & RAG에 대해 더 알고 싶다면 마지막 부록 섹션으로 이동하세요.

---

## 예시 실습장

이 곳은 이 수업에서 보여준 프롬프트 예시들을 자유롭게 실험해보고 프롬프트를 조정하여 Claude의 응답이 어떻게 달라지는지 확인할 수 있는 공간입니다.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)